In [2]:
import re
import pandas as pd
from tqdm import tqdm

In [3]:
regex = "([0-9]*\.?[0-9]*)(?: *)?(\( ([0-9]*)(?: *)?([a-z]*) \))?(?: *)?([a-z]*)(?: *)?(?:of )?(.*)"

test_regex = re.compile(regex)

In [4]:
import nltk
gram = ("NP: {<VB.*|JJ>?<N.*>*}")

In [5]:
df_INQ = pd.read_csv('df_INQ_test_cleaned.csv')
df_ingredients = pd.read_csv('df_ingredients_clean.csv')

df_INQ.drop(columns=['Unnamed: 0'], inplace= True)
df_ingredients.drop(columns=['Unnamed: 0'], inplace= True)

df_INQ_agged = df_INQ.groupby('ID', as_index=False).agg(pd.Series.tolist)
df_ingredients_agged = df_ingredients.groupby('ID', as_index=False).agg(pd.Series.tolist)

In [14]:
units = ['assortment',
 'bag',
 'bags',
 'ball',
 'balls',
 'bar',
 'bars',
 'basket',
 'baskets',
 'batch',
 'blades',
 'block',
 'bone',
 'bottle',
 'bottles',
 'bowl',
 'bowls',
 'box',
 'boxes',
 'bulb',
 'bulbs',
 'bunch',
 'bunches',
 'can',
 'canister',
 'cans',
 'carton',
 'cartons',
 'chunk',
 'chunks',
 'clove',
 'cloves',
 'container',
 'containers',
 'cube',
 'cubes',
 'cup',
 'cups',
 'dash',
 'dashes',
 'drop',
 'drops',
 'ear',
 'ears',
 'flatout',
 'fluid ounces',
 'foot',
 'g',
 'gallon',
 'gallons',
 'gram',
 'grams',
 'handful',
 'head',
 'heads',
 'inch',
 'inches',
 'jar',
 'jars',
 'kg',
 'knob',
 'lb',
 'leaf',
 'leaves',
 'length',
 'lengths',
 'liter',
 'liters',
 'loaf',
 'loaves',
 'log',
 'logs',
 'lump',
 'mg',
 'milliliter',
 'minute',
 'ml',
 'neck',
 'ounce',
 'ounces',
 'oval',
 'oz',
 'package',
 'packaged',
 'packages',
 'packet',
 'packets',
 'peel',
 'peels',
 'piece',
 'pieces',
 'pinch',
 'pinches',
 'pint',
 'pints',
 'pitted',
 'pkg',
 'pocket',
 'pod',
 'pods',
 'portions',
 'pot',
 'pouch',
 'pouches',
 'pound',
 'pounds',
 'quart',
 'quarts',
 'rack',
 'racks',
 'ring',
 'rings',
 'roll',
 'rolls',
 'round',
 'scoop',
 'scoops',
 'serving',
 'servings',
 'set',
 'sheet',
 'sheets',
 'shells',
 'shot',
 'shots',
 'skewers',
 'slab',
 'slabs',
 'sleeve',
 'sleeves',
 'slice',
 'slices',
 'spears',
 'splash',
 'splashes',
 'spray',
 'sprays',
 'sprinkle',
 'square',
 'squares',
 'squeeze',
 'squeezes',
 'squirts',
 'stalk',
 'stalks',
 'star',
 'stems',
 'stewing',
 'stick',
 'sticks',
 'store',
 'strands',
 'straw',
 'strip',
 'strips',
 'tablespoon',
 'tablespoons',
 'tablet',
 'tablets',
 'teaspoon',
 'teaspoons',
 'tins',
 'tray',
 'trays',
 'tub',
 'tube',
 'tubes',
 'tubs',
 'twist',
 'twists',
 'wedge',
 'wedges',
 'wheel',
 'wheels',
 'wrap',
 'wraps',
 'tbsp',
 'tsp',
 'l',
 'g',
 'tbsps',
 'tsps',
 'cm',
 'tin']

In [8]:
df_ingredients_with_notes = pd.read_csv('ingredients_PCA_with_notes.csv')
df_ingredients_with_notes.drop(columns=['Unnamed: 0'], inplace= True)

In [9]:
df_ingredients_with_notes

,ID,Ingredients,match,order,note
0,1,gochujang base,gochujang base,2,Perfect
1,1,ketchup,ketchup,1,Perfect
2,1,maple syrup,maple syrup,2,Perfect
3,1,soy sauce,soy sauce,2,Perfect
4,1,seasoned rice wine vinegar,seasoned rice wine vinegar,4,Perfect
...,...,...,...,...,...
26173,902,banana,banana,1,Perfect
26174,902,almond milk,almond milk,2,Perfect
26175,902,packet frozen acai,za'atar,0,Not perfect
26176,903,banana,banana,1,Perfect


In [28]:
import json

with open('tree.json') as json_file:
    dict_tree = json.load(json_file)

In [31]:
from strsimpy import Cosine
from itertools import permutations

cos = Cosine(1)

def find(s, trees, order, steps):
    max_order = s.split(' ')
    candidates = []
    
    if order < len(s.split(' ')):
        
        for i in permutations(s.split(' '), order):
            for tree in trees:
                candidate = tree.get(' '.join(i))
                if candidate is not None:
                    steps.append((' '.join(i), order))
                    candidates.append(candidate)
 
        if not candidates:
            
            if any([not(tree.keys()) for tree in trees]):
                
                return steps[-1]
            
            max_cosine_score = float('-inf')
            
            for i in permutations(s.split(' '), order):
                for tree in trees:
                    candidates = tree.keys()
                    for key in candidates:
                        if 1 - cos.distance(' '.join(i), key) > max_cosine_score:
                            max_cosine_score = 1 - cos.distance(s, key)
                            best_key = key
                steps.append((best_key, order - 1))
            
            return steps[-1]
            
        find(s, candidates, order+1, steps)
        
    else:
        for tree in trees:
            candidate = tree.get(s)
            if candidate is not None:
                steps.append((s, order))
                
        if not steps:
            
            max_cosine_score = float('-inf')
            
            for tree in trees:
                candidates = tree.keys()
                for key in candidates:
                    if 1 - cos.distance(s, key) > max_cosine_score:
                        max_cosine_score = 1 - cos.distance(s, key)
                        best_key = key
                            
            steps.append((best_key, order - 1))
            
            return steps[-1]
            
    return steps[-1]

In [5]:
def loose_eval(s, l):
    for i in l:
        if s in i or i in s:
            return True
    return False

In [40]:
diff = []
stat_result = []

for row in tqdm(df_INQ_agged.itertuples()):
    
    additional = []
    perfect_match_from_database = 0
    perfect_match_from_dict = 0
    perfect_match_from_database_not_in_dict = 0
    loose_match_from_database = 0
    loose_match_from_dict = 0
    loose_match_from_dict_predict = 0
    wrong_count = 0
    
    ID = row.ID
    
    ingredients = df_ingredients_with_notes[df_ingredients_with_notes['ID'] == ID]
    
    base_match = ingredients['Ingredients'].to_list()
    
    perfect_match = ingredients[ingredients['note'] == 'Perfect']['match'].to_list()
    
    not_agree_match = ingredients[ingredients['note'] == 'Not perfect']['match'].to_list()
    
    input_texts = row.INQs
    
    for text in input_texts:
        
        result = test_regex.match(text)
    
        if result is not None and result.groups()[-2] in units:
            
            if result.groups()[-1] != '':

                result = result.groups()[-1]
            
            else:
                
                result = text
                
        else:
                
            result = text

        chunking = nltk.RegexpParser(gram)
        sent_token = nltk.word_tokenize(result)
        tagging = nltk.pos_tag(sent_token)
        tree = chunking.parse(tagging)
        
        result = []

        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                result.extend(subtree.leaves())
                
        final = " ".join([x[0] for x in result])
        
        if final is not None:
            if final in perfect_match: # perfect match
                perfect_match_from_database += 1
            elif find(final, [dict_tree], 1, []):
                if find(final, [dict_tree], 1, [])[0] == final:
                    perfect_match_from_dict += 1
                elif 1 - cos.distance(final, find(final, [dict_tree], 1, [])[0]) > 0.9:
                    loose_match_from_dict += 1
                else:
                    diff.append(f'{text}, {final}, {ID} \n')
                    wrong_count += 1
            elif final in base_match:
                perfect_match_from_database_not_in_dict += 1
                additional.append(final)
            elif final in loose_eval(final, base_match): # ingredients not 100 found in dict so default to base
                loose_match_from_database += 1
            elif final in loose_eval(final, not_agree_match):
                loose_match_from_dict_predict += 1
            elif find(final, [dict_tree], 1, []):
                if 1 - cos.distance(final, find(final, [dict_tree], 1, [])[0]) > 0.9:
                    loose_match_from_dict += 1
                else:
                    diff.append(f'{text}, {final}, {ID} \n')
                    wrong_count += 1
            else:
                diff.append(f'{text}, {final}, {ID} \n')
                wrong_count += 1
            # extract.append(final)
            
    stat_result.append({
        'additional': additional,
        'perfect_match_from_database' : perfect_match_from_database,
        'perfect_match_from_dict': perfect_match_from_dict,
        'perfect_match_from_database_not_in_dict': perfect_match_from_database_not_in_dict,
        'loose_match_from_database': loose_match_from_database,
        'loose_match_from_dict': loose_match_from_dict,
        'loose_match_from_dict_predict': loose_match_from_dict_predict,
        'wrong_count': wrong_count,
    })
        
file = open('compare.txt', 'w')

file.writelines(diff)

file.close()

2299it [03:14, 11.81it/s]


In [41]:
stat_result[0]

{'additional': [],
 'perfect_match_from_database': 15,
 'perfect_match_from_dict': 3,
 'perfect_match_from_database_not_in_dict': 0,
 'loose_match_from_database': 0,
 'loose_match_from_dict': 2,
 'loose_match_from_dict_predict': 0,
 'wrong_count': 3}

In [47]:
analysis = []

for i in stat_result:
    total = sum(list(i.values())[1:])
    strict_count = i['perfect_match_from_database'] + i['perfect_match_from_dict'] + i['perfect_match_from_database_not_in_dict']
    loose_count = total - i['wrong_count']
    strict_acc = strict_count / total
    loose_acc = loose_count / total
    analysis.append((strict_acc, loose_acc))

In [49]:
strict_accs = [x[0] for x in analysis]
loose_accs = [x[1] for x in analysis]

In [50]:
import numpy as np

In [51]:
print('Strict mean: ', np.mean(strict_accs))
print('Strict std: ', np.std(strict_accs))
print('Strict var: ', np.var(strict_accs))
print('Loose mean: ', np.mean(loose_accs))
print('Loose std: ', np.std(loose_accs))
print('Loose var: ', np.var(loose_accs))

Strict mean:  0.7100545710010074
Strict std:  0.16769228166825462
Strict var:  0.028120701331105245
Loose mean:  0.7726803903229852
Loose std:  0.15111956608105545
Loose var:  0.022837123252526483


In [38]:
len(df_INQ_agged.loc[0]['INQs'])

23